In [6]:
import os
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [40]:
lst_sq_trains = ['MOT17-02-FRCNN', 'MOT17-04-FRCNN', 'MOT17-05-FRCNN', 'MOT17-09-FRCNN', 'MOT17-10-FRCNN', 'MOT17-11-FRCNN']
# lst_sq_trains = ['MOT17-02-DPM']
root_dir = './data/MOT17/train'
gt = 'gt/gt.txt'
save_path = './data/person_reid/train'
os.makedirs('./data/person_reid/train', exist_ok = True)


for idx, folder_name in enumerate(lst_sq_trains):
    with open(os.path.join(root_dir, folder_name, gt), 'r') as f:
        progress_bar = tqdm(f.readlines(), desc=f"Folder {folder_name}", leave=False)
        for line in progress_bar:
            img_name = '0'*(6-len(line.split(',')[0])) + line.split(',')[0] + '.jpg'
            img_id = line.split(',')[1]
            img = cv2.imread(f'{root_dir}/{folder_name}/img1/{img_name}')
            left = int(line.split(',')[2])
            top = int(line.split(',')[3])
            width = int(line.split(',')[4])
            height = int(line.split(',')[5])
            im_crop = img[top: top+height,left: left+width]
            try:
                cv2.imwrite(f'{save_path}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
            except:
                continue
        #     plt.imshow(im_crop)
        #     plt.show()
        #     break
        # break

In [6]:
import numpy as np
x = np.array([1, 2, 3, 4])
x[2:] = x[:2] + x[2:]
x

array([1, 2, 4, 6])

In [39]:
val_seq = ['MOT17-13-FRCNN']
root_dir = './data/MOT17/train'
gt = 'gt/gt.txt'
save_path_query = './data/person_reid/val/query'
save_path_gallery = './data/person_reid/val/gallery'
os.makedirs(save_path_query, exist_ok = True)
os.makedirs(save_path_gallery, exist_ok = True)

for idx, folder_name in enumerate(val_seq):
    with open(os.path.join(root_dir, folder_name, gt), 'r') as f:
        lines = f.readlines()
        idx_arr = np.arange(len(lines))
        idx_random_20_percent = np.random.choice(idx_arr, size=int(0.2 * len(idx_arr)), replace=False)
        progress_bar = tqdm(lines, desc=f"Folder {folder_name}", leave=False)

        for idx, line in enumerate(progress_bar):
            img_name = '{:06d}'.format(int(line.split(',')[0])) + '.jpg'
            img_id = line.split(',')[1]
            img = cv2.imread(f'{root_dir}/{folder_name}/img1/{img_name}')
            left = int(line.split(',')[2])
            top = int(line.split(',')[3])
            width = int(line.split(',')[4])
            height = int(line.split(',')[5])
            im_crop = img[top: top+height,left: left+width]
            
            if idx in idx_random_20_percent:
                try:
                    cv2.imwrite(f'{save_path_query}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
                except:
                    continue
            else:
                try:
                    cv2.imwrite(f'{save_path_gallery}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
                except:
                    continue